In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split

In [2]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")

In [3]:
train.drop(["ID"], axis=1, inplace=True)

In [36]:
train_set,test_set=train_test_split(train,test_size=0.1,random_state=42)
print(train_set.shape,test_set.shape)

(876095, 7) (97344, 7)


In [37]:
train_label=train_set["global_horizontal_irradiance"].copy()
train_set.drop(['global_horizontal_irradiance'], axis=1, inplace=True)
print(train_set.shape,train_label.shape)

(876095, 6) (876095,)


C:\Users\USER\anaconda3\envs\final_gpu\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [38]:
def processing(train_data):
    from sklearn.preprocessing import StandardScaler
    scaled_features = StandardScaler().fit_transform(train_data.values)
    scaled_data=pd.DataFrame(scaled_features,index=train_data.index,columns=train_data.columns)

    return scaled_data
training_data=train_set
train_data=processing(train_set)

print(train_data.head)

<bound method NDFrame.head of          precipitation   atmospheric_pressure  relative_humidity  \
765123        -0.098972             -0.697778          -0.542181   
290301        -0.098972              0.138365           0.107788   
570959        -0.098972              0.844729           0.135572   
193964        -0.098972             -0.269510          -0.684635   
765692        -0.098972              0.750176          -1.621655   
...                 ...                   ...                ...   
259178        -0.098972              1.293391           0.775686   
365838        -0.098972              0.718659          -0.527212   
131932        -0.098972             -0.977728           0.589913   
671155        -0.098972             -1.337956           0.775686   
121958        -0.098972              1.083892          -2.314389   

        air_temperature    wind_direction  wind_speed  
765123          1.137656        -0.113849    0.153521  
290301         -0.013885         1.555793

In [39]:
y_valid=test_set["global_horizontal_irradiance"].copy()
test_set.drop(['global_horizontal_irradiance'], axis=1, inplace=True)
X_valid=test_set
print(X_valid.shape,y_valid.shape)

(97344, 6) (97344,)


C:\Users\USER\anaconda3\envs\final_gpu\lib\site-packages\pandas\core\frame.py:4164: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [40]:
print(train_data.shape,X_valid.shape)

(876095, 6) (97344, 6)


In [42]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
forest_reg=RandomForestRegressor()
boost_reg=GradientBoostingRegressor()


In [23]:
%%time
forest_reg.fit(X_train,train_label)

Wall time: 14min 2s


RandomForestRegressor(n_jobs=-1)

In [43]:
from sklearn.model_selection import RandomizedSearchCV
from pprint import pprint

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 1000, num = 8)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 60, num = 4)]
# Minimum number of samples required to split a node
min_samples_split = [32,64,100,200,300]
# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
random_grid = {'n_estimators': [30, 80, 100, 200, 400,600],
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_leaf': [1, 2, 4],
               'min_samples_split': min_samples_split,
               'bootstrap': bootstrap
              }

pprint(random_grid)

{'bootstrap': [True, False],
 'max_depth': [1, 20, 40, 60],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [32, 64, 100, 200, 300],
 'n_estimators': [30, 80, 100, 200, 400, 600]}


In [44]:
%%time
# Use the random grid to search for best hyperparameters
# First create the base model to tune
rf = RandomForestRegressor()
# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid,scoring='neg_mean_squared_error', n_iter = 30, cv = 3, verbose=2, random_state=42, n_jobs = -1)
# Fit the random search model
rf_random.fit(train_data, train_label)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  25 tasks      | elapsed: 104.4min
[Parallel(n_jobs=-1)]: Done  90 out of  90 | elapsed: 195.5min finished


KeyboardInterrupt: 

In [47]:
rf_random.best_params_

{'n_estimators': 600,
 'min_samples_split': 32,
 'min_samples_leaf': 4,
 'max_features': 'auto',
 'max_depth': 60,
 'bootstrap': True}

In [50]:
final_model = rf_random.best_estimator_

In [53]:
from sklearn.metrics import mean_squared_error

predictions = final_model.predict(train_data)
basic_mse=mean_squared_error(train_label,predictions)
basic_mse=np.sqrt(basic_mse)
print(basic_mse)

IndexError: list index out of range